<a href="https://colab.research.google.com/github/pnjha/NLP-Assignments/blob/master/s2sattn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/My\ Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive


In [0]:
%tensorflow_version 1.14
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
import copy
import math
import tensorflow as tf
import keras as keras
from keras.optimizers import SGD
from keras.initializers import RandomUniform
# from keras.models import Sequential
# from keras.layers import LSTM
# from keras.layers import Dense
# from keras.layers import TimeDistributed, Bidirectional, Embedding
# from keras.layers import RepeatVector
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, Model, load_model
# from keras.layers import Dense, TimeDistributed
from tensorflow.keras.layers import Embedding, Dense, LSTM, CuDNNLSTM, Input, Embedding, TimeDistributed, Flatten, Dropout, Activation, dot, concatenate
# from keras.layers import 
# from keras.layers import 
from keras.callbacks import ModelCheckpoint
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

`%tensorflow_version` only switches the major version: `1.x` or `2.x`.
You set: `1.14`. This will be interpreted as: `1.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.


In [0]:
X = pd.read_table('./NLA S20 - Assignment 2 Data/enghin/train.en', names=['src'])
Y = pd.read_table('./NLA S20 - Assignment 2 Data/enghin/train.hi', names=['dest'])
lines = pd.concat([X[:1000],Y[:1000]], axis=1)
print(len(lines))
# lines = shuffle(lines)
lines.sample(10)

1000


,src,dest
452,Under the National Maternity Benefit Scheme ab...,राष्ट्रीय मातृत्व लाभ योजना के अन्तर्गत गरीबी ...
587,When any food item or beverage with salmonella...,जब सालमोनेला टिफी युक्त कोई भी खाद्य पदार्थ या...
282,Primary vaccination keeps a person safe for a ...,प्राथमिक टीकाकरण व्यक्ति को एक वर्ष तक सुरक्षि...
354,Leprosy is a bacteria born contagious disease .,कुष्ठ रोग एक जीवाणु से उत्पन्न होने वाला छूत क...
811,By doing regular exercises also the muscles ar...,नियमित व्यायाम करने से भी जोड़ों के आसपास की मा...
826,Knot increasing rapidly .,गाँठ का तेजी से बढ़ना ।
112,The most credible method of finding out the pr...,डॉक्टर द्वारा पुरस्थ ग्रंथी के कैंसर का आंरभिक...
500,Dead birth due to inborn syphilis .,जन्मजात उपदंश ( सिफिलिस ) के कारण बच्चे का मृत...
330,The goal of health programs is to increase the...,स्वास्थ्य कार्यक्रमों का लक्ष्य दम्पति सुरक्षा...
79,Any wound that do not heal in enough time .,कोई भी घाव जो काफी समय से भरता नहीं हो ।


In [0]:
def process_data(data,append_char):
  data = data.apply(lambda x: x.lower())
  data = data.apply(lambda x: x.strip())
  data = data.apply(lambda x: re.sub("'", '', x))
  exclude = set(string.punctuation) # Set of all special characters
  data = data.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
  remove_digits = str.maketrans('', '', digits)
  data = data.apply(lambda x: x.translate(remove_digits))
  # if append_char:
    # data = data.apply(lambda x : '<SOS> '+ x + ' <EOS>')
  return data

In [0]:
lines.src = process_data(lines.src,False)
lines.dest = process_data(lines.dest,True)
# lines.dest.sample(10)

In [0]:
def get_vocab_list(data):
  vocab = []
  for line in data:
    temp = line.split()
    for item in temp:
      if item not in vocab:
        vocab.append(item)
  # print(vocab)
  return vocab

In [0]:
src_vocab = get_vocab_list(lines.src)
dest_vocab = get_vocab_list(lines.dest)

In [0]:
print(src_vocab[:10])
print(dest_vocab[:10]) 

['the', 'treatment', 'of', 'cataract', 'is', 'possible', 'through', 'surgery', 'only', 'complete']
['मोतियाबिंद', 'का', 'उपचार', 'केवल', 'शल्यचिकित्सा', 'द्वारा', 'ही', 'सम्भव', 'है', '।']


In [0]:
# Max Length of source sequence
def get_max_sentence_size(data):
  max_len = 0
  for line in data:
    temp = line.split()
    max_len = max(max_len, len(temp))
  return max_len

In [0]:
src_max_len = get_max_sentence_size(lines.src)
dest_max_len = get_max_sentence_size(lines.dest)
src_max_len,dest_max_len

(57, 57)

In [0]:
src_vocab_len = len(src_vocab)
dest_vocab_len = len(dest_vocab) + 1
src_vocab_len, dest_vocab_len

(2538, 3216)

In [0]:
def get_word_to_index_dict(vocab):
  word_to_index = {}
  for i, word in enumerate(vocab):
    word_to_index[word] = i
  return word_to_index

In [0]:
src_word_to_index = get_word_to_index_dict(src_vocab)
dest_word_to_index = get_word_to_index_dict(dest_vocab)

In [0]:
print(src_word_to_index)
print(dest_word_to_index)

{'the': 0, 'treatment': 1, 'of': 2, 'cataract': 3, 'is': 4, 'possible': 5, 'through': 6, 'surgery': 7, 'only': 8, 'complete': 9, 'lens': 10, 'capsule': 11, 'taken': 12, 'out': 13, 'in': 14, 'method': 15, 'intra': 16, 'capsular': 17, 'extraction': 18, 'after': 19, 'operation': 20, 'spectacles': 21, 'are': 22, 'used': 23, 'posttest': 24, 'during': 25, 'implanted': 26, 'at': 27, 'front': 28, 'iris': 29, 'frontal': 30, 'part': 31, 'eye': 32, 'extra': 33, 'and': 34, 'whole': 35, 'matter': 36, 'extracted': 37, 'posterior': 38, 'left': 39, 'intact': 40, 'fitted': 41, 'bag': 42, 's': 43, 'lrb': 44, 'small': 45, 'incision': 46, 'rrb': 47, 'mm': 48, 'tunnel': 49, 'formed': 50, 'while': 51, 'rest': 52, 'follows': 53, 'like': 54, 'that': 55, 'ecceiol': 56, 'no': 57, 'stitches': 58, 'applied': 59, 'black': 60, 'nerves': 61, 'dilapidates': 62, 'gradually': 63, 'it': 64, 'happens': 65, 'due': 66, 'to': 67, 'water': 68, 'pressure': 69, 'or': 70, 'because': 71, 'not': 72, 'enough': 73, 'blood': 74, 're

In [0]:
def get_index_to_word_dict(vocab):
  index_to_word = {}
  for i, word in enumerate(vocab):
    index_to_word[i] = word
  return index_to_word  

In [0]:
src_index_to_word = get_index_to_word_dict(src_vocab)
dest_index_to_word = get_index_to_word_dict(dest_vocab)

In [0]:
print(src_index_to_word)
print(dest_index_to_word)

{0: 'the', 1: 'treatment', 2: 'of', 3: 'cataract', 4: 'is', 5: 'possible', 6: 'through', 7: 'surgery', 8: 'only', 9: 'complete', 10: 'lens', 11: 'capsule', 12: 'taken', 13: 'out', 14: 'in', 15: 'method', 16: 'intra', 17: 'capsular', 18: 'extraction', 19: 'after', 20: 'operation', 21: 'spectacles', 22: 'are', 23: 'used', 24: 'posttest', 25: 'during', 26: 'implanted', 27: 'at', 28: 'front', 29: 'iris', 30: 'frontal', 31: 'part', 32: 'eye', 33: 'extra', 34: 'and', 35: 'whole', 36: 'matter', 37: 'extracted', 38: 'posterior', 39: 'left', 40: 'intact', 41: 'fitted', 42: 'bag', 43: 's', 44: 'lrb', 45: 'small', 46: 'incision', 47: 'rrb', 48: 'mm', 49: 'tunnel', 50: 'formed', 51: 'while', 52: 'rest', 53: 'follows', 54: 'like', 55: 'that', 56: 'ecceiol', 57: 'no', 58: 'stitches', 59: 'applied', 60: 'black', 61: 'nerves', 62: 'dilapidates', 63: 'gradually', 64: 'it', 65: 'happens', 66: 'due', 67: 'to', 68: 'water', 69: 'pressure', 70: 'or', 71: 'because', 72: 'not', 73: 'enough', 74: 'blood', 75:

In [0]:
def reverse_train_sentences(data):
  l = data.tolist()
  t = []
  for item in l:
    p = item.strip().split()
    p.reverse()
    k = []
    for i in p:
      i = i.strip()
      if len(i) > 0:
        k.append(i)
    p = k
    p = " ".join(p)
    p = p.strip()
    t.append(p)
  df = pd.DataFrame({'data':t})
  return df.data  

In [0]:
def transform(mapping, data, sentence_size):
    transformed_data = np.zeros(shape=(len(data), sentence_size), dtype='int')
    for i in range(len(data)):
        l = data[i].split()
        for j in range(len(l)):
            transformed_data[i][j] = mapping[l[j]]
    return transformed_data

In [0]:
# Train - Test Split
X, y = lines.src.values, lines.dest.values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1,shuffle=False)
X_train = reverse_train_sentences(X_train)

encoded_training_input = transform(src_word_to_index, X_train, src_max_len)
encoded_validation_input = transform(src_word_to_index, X_test, src_max_len)
encoded_training_output = transform(dest_word_to_index, y_train, dest_max_len)
encoded_validation_output = transform(dest_word_to_index, y_test, dest_max_len)

print('encoded_training_input', encoded_training_input.shape)
print('encoded_training_output', encoded_training_output.shape)
print('encoded_validation_input', encoded_validation_input.shape)
print('encoded_validation_output', encoded_validation_output.shape)
# X_train_normal = copy.deepcopy(X_train)
# print(X_train.shape, X_test.shape)
# X_train.sample(10)

encoded_training_input (900, 57)
encoded_training_output (900, 57)
encoded_validation_input (100, 57)
encoded_validation_output (100, 57)


In [0]:
latent_dim = 200
embedding_dim = 300

In [0]:
encoder_input = Input(shape=(src_max_len,))
decoder_input = Input(shape=(dest_max_len,))
encoder = Embedding(src_vocab_len, embedding_dim, input_length=src_max_len, mask_zero=True)(encoder_input)
encoder = LSTM(latent_dim, return_sequences=True, unroll=True)(encoder)
encoder_last = encoder[:,-1,:]

print('encoder', encoder)
print('encoder_last', encoder_last)

decoder = Embedding(dest_vocab_len, embedding_dim, input_length=dest_max_len, mask_zero=True)(decoder_input)
decoder = LSTM(latent_dim, return_sequences=True, unroll=True)(decoder, initial_state=[encoder_last, encoder_last])

print('decoder', decoder)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
encoder Tensor("lstm/transpose_2:0", shape=(?, 57, 200), dtype=float32)
encoder_last Tensor("strided_slice_1:0", shape=(?, 200), dtype=float32)
decoder Tensor("lstm_1_1/transpose_2:0", shape=(?, 57, 200), dtype=float32)


In [0]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    hidden_with_time_axis = tf.expand_dims(query, 1)

    score = self.V(tf.nn.tanh(self.W1(values) + self.W2(hidden_with_time_axis)))
    attention_weights = tf.nn.softmax(score, axis=1)
    context_vector = attention_weights * values

    return context_vector, attention_weights


In [0]:
# attention = dot([decoder, encoder], axes=[2, 2])
# attention = Activation('softmax', name='attention')(attention)
# # print('attention', attention)
# # print('attention shape', attention.shape)
# context = dot([attention, encoder], axes=[2,1])
# # print('context', context)
# # print('context', context.shape)

attention_layer = BahdanauAttention(100)
context, attention_weights = attention_layer(encoder_last,encoder)
# context = dot([attention_weights, encoder], axes=[2,1])
print('context', context)

decoder_combined_context = concatenate([context, decoder])
print('decoder_combined_context', decoder_combined_context)

output = TimeDistributed(Dense(latent_dim, activation="tanh"))(decoder_combined_context)
output = TimeDistributed(Dense(dest_vocab_len, activation="softmax"))(output)
print('output', output)

context Tensor("bahdanau_attention_3/mul:0", shape=(?, 57, 200), dtype=float32)
decoder_combined_context Tensor("concatenate/concat:0", shape=(?, 57, 400), dtype=float32)
output Tensor("time_distributed_1/Reshape_1:0", shape=(?, 57, 3216), dtype=float32)


In [0]:
model = Model(inputs=[encoder_input, decoder_input], outputs=[output])
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 57)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 57, 300)      761400      input_1[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     (None, 57, 200)      400800      embedding[0][0]                  
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 57)]         0                                            
______________________________________________________________________________________________

In [0]:
training_encoder_input = encoded_training_input
training_decoder_input = np.zeros_like(encoded_training_output)
training_decoder_input[:, 1:] = encoded_training_output[:,:-1]
training_decoder_input[:, 0] = 1
training_decoder_output = np.eye(dest_vocab_len)[encoded_training_output.astype('int')]

validation_encoder_input = encoded_validation_input
validation_decoder_input = np.zeros_like(encoded_validation_output)
validation_decoder_input[:, 1:] = encoded_validation_output[:,:-1]
validation_decoder_input[:, 0] = 1
validation_decoder_output = np.eye(dest_vocab_len)[encoded_validation_output.astype('int')]

In [0]:
model.fit(x=[training_encoder_input, training_decoder_input], y=[training_decoder_output],
          validation_data=([validation_encoder_input, validation_decoder_input], [validation_decoder_output]),
          batch_size=64, epochs=50)

# model.save('model.h5')

Train on 900 samples, validate on 100 samples
Epoch 1/50
900/900 [==============================] - 26s 29ms/sample - loss: 1.6185 - acc: 0.1840 - val_loss: 2.3430 - val_acc: 0.1632
Epoch 2/50
900/900 [==============================] - 26s 29ms/sample - loss: 1.5935 - acc: 0.1876 - val_loss: 2.4572 - val_acc: 0.1546
Epoch 3/50
900/900 [==============================] - 26s 29ms/sample - loss: 1.5708 - acc: 0.1916 - val_loss: 2.2446 - val_acc: 0.1568
Epoch 4/50
900/900 [==============================] - 26s 29ms/sample - loss: 1.5514 - acc: 0.1995 - val_loss: 2.3550 - val_acc: 0.1637
Epoch 5/50
900/900 [==============================] - 26s 29ms/sample - loss: 1.5287 - acc: 0.2021 - val_loss: 2.3727 - val_acc: 0.1616
Epoch 6/50
900/900 [==============================] - 26s 29ms/sample - loss: 1.5081 - acc: 0.2090 - val_loss: 2.4673 - val_acc: 0.1621
Epoch 7/50
900/900 [==============================] - 26s 29ms/sample - loss: 1.4895 - acc: 0.2135 - val_loss: 2.4508 - val_acc: 0.1637
Ep

In [0]:
def calculate_bleu_score(actual_string, predicted_string):
  actual_string = copy.deepcopy(actual_string)
  predicted_string = copy.deepcopy(predicted_string)
  reference = re.split("\s",actual_string.strip())
  candidate = re.split("\s",predicted_string.strip())
  smoothie = SmoothingFunction().method3
  score = sentence_bleu(reference, candidate, smoothing_function=smoothie)
  return score

In [42]:
def generate(text):
    encoder_input = transform(src_word_to_index, [text.lower()], src_max_len)
    decoder_input = np.zeros(shape=(len(encoder_input), dest_max_len))
    decoder_input[:,0] = 1
    for i in range(1, dest_max_len):
        output = model.predict([encoder_input, decoder_input]).argmax(axis=2)
        decoder_input[:,i] = output[:,i]
    return decoder_input[:,1:]

def decode(decoding, sequence):
    text = ''
    l = []
    for i in sequence:
        if i == 0:
            break
        l.append(decoding[i])
    text = " ".join(l)
    return text

def predict_dest(text):
    decoder_output = generate(text)
    return decode(dest_index_to_word, decoder_output[0])

bleu_score = 0.0
for i, line in enumerate(X_test):
  print("Actual Source:",line)
  print("Actual Target: ",y_test[i])
  dest_predicted = predict_dest(line)
  print("Predicted: ",dest_predicted)
  curr_bs = calculate_bleu_score(y_test[i],dest_predicted)
  bleu_score += curr_bs
  print("Current BLEU Score: ",curr_bs)
  # break
print("Overall BLEU Score: ", bleu_score/len(xtest[0]))

Actual Source: if eaten less chapatti at dinner and eaten rice daily then this light food can keep your health good 
Actual Target:  यदि रात्रि के भोजन में रोटी कम खाएँ और चावल प्रतिदिन खाएँ तो यह हलक़ा भोजन आपका स्वास्थ्य ठीक रखेगा ।
Predicted:  पीतज्वर के कारण श्‍वास फैलाने के लिये की मात्रा में परेशानी होती है ।
Current BLEU Score:  0.028398387225677897
Actual Source: rice is acidity killer  easily digestible  that which increases blood in body and is proper diet in diarrhoea and dysentery 
Actual Target:  चावल मंदाग्नि नाशक  सुपाच्य  शरीर में खून बढ़ाने वाला  शीघ्र पचने वाला  अतिसार व पेचिश में पथ्य भोजन है ।
Predicted:  पीतज्वर के अन्तर्गत भारत सरकार से भारत सरकार से चलाया जा सकता है और फिर पैर की आवश्यकता है ।
Current BLEU Score:  0.019146030690102515
Actual Source: three year old rice is very tasteful and that which produces glow 
Actual Target:  तीन साल पुराना चावल काफी स्वादिष्ट व ओजवर्धक होता है ।
Predicted:  पीतज्वर के अन्तर्गत भारत सरकार से से भी क्षमता से भी व्यायाम करने की 

NameError: ignored